<a href="https://colab.research.google.com/github/jdmartinev/ArtificialIntelligenceIM/blob/main/Lecture07/notebooks/L07_GANS_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Generativas Antagónicas (GAN)

El objetivo principal de una **Red Generativa Antagónica** (GAN, por sus siglas en inglés) es generar imágenes que sean similares (pero no idénticas) a las del conjunto de datos de entrenamiento.

Una GAN está compuesta por dos redes neuronales que se entrenan en conjunto, compitiendo entre sí:

* **Generador**: toma un vector aleatorio y genera una imagen a partir de él.
* **Discriminador**: es una red neuronal que debe distinguir entre una imagen original (del conjunto de datos de entrenamiento) y una imagen generada por el generador.

Durante el entrenamiento, el **generador** mejora su capacidad para engañar al **discriminador**, mientras que el discriminador se vuelve más hábil en diferenciar imágenes reales de las falsas. El objetivo es alcanzar un equilibrio en el que el generador produzca imágenes tan realistas que el discriminador no pueda distinguirlas de las imágenes reales.

![Imagen de la arquitectura del VAE](https://drive.google.com/uc?id=1EJjgHmq8xJbAhdbMYWWTumM7OHQG8oZp)

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import numpy as np

## Generador

El rol del generador es tomar un vector aleatorio de cierto tamaño (similar al vector latente en los autoencoders) y generar la imagen objetivo. El proceso es muy parecido al lado generativo de un autoencoder.

En nuestro ejemplo, utilizaremos redes neuronales lineales y el conjunto de datos MNIST para generar imágenes de dígitos escritos a mano. El generador comenzará con un vector de entrada aleatorio y aprenderá a transformarlo en imágenes que se asemejen a los dígitos reales del conjunto de datos.


In [3]:
latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),  # Add a comma after latent_dim
        Dense(256),  # First Dense layer with input shape (equivalent to 256 neurons)
        LeakyReLU(negative_slope=0.2),  # Add LeakyReLU separately
        BatchNormalization(momentum=0.8),  # Batch Normalization

        Dense(512),  # Second Dense layer
        LeakyReLU(negative_slope=0.2),  # LeakyReLU activation
        BatchNormalization(momentum=0.8),  # Batch Normalization

        Dense(1024),  # Third Dense layer
        LeakyReLU(negative_slope=0.2),  # LeakyReLU activation
        BatchNormalization(momentum=0.8),  # Batch Normalization

        Dense(784, activation='tanh'),  # Output layer
        Reshape((28, 28))  # Reshape to (28,28)
    ],
    name = "generator"
)

generator.summary()

Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 784)                 │         803,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 28, 28)              │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,500,688 (5.72 MB)

 Trainable params: 1,497,104 (5.71 MB)

 Non-trainable params: 3,584 (14.00 KB)

## Algunos trucos utilizados en el generador:

* En lugar de **ReLU**, utilizamos **LeakyReLU**, es decir, una variación de ReLU que no es exactamente 0 para valores negativos de $x$, sino que aplica una función lineal con una pendiente muy pequeña.
* Utilizamos **BatchNorm1D** para estabilizar el entrenamiento, lo que ayuda a normalizar las activaciones y acelerar la convergencia.
* La función de activación en la última capa es **Tanh**, lo que significa que la salida estará en el rango [-1,1].


## Discriminador

El discriminador es una red clásica de clasificación de imágenes. En nuestro primer ejemplo, también utilizaremos un clasificador lineal.


In [5]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28)),  # Input layer with shape (28,28)
        Flatten(),  # Flatten the input

        Dense(784),  # First Dense layer with 784 units
        LeakyReLU(negative_slope=0.2),  # LeakyReLU activation

        Dense(784 // 2),  # Second Dense layer with 392 units (784/2)
        LeakyReLU(negative_slope=0.2),  # LeakyReLU activation

        Dense(1, activation='sigmoid')  # Output layer with sigmoid activation
    ],
    name = "discriminator"
)

discriminator.summary()

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 784)                 │         615,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_8 (LeakyReLU)            │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 392)                 │         307,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_9 (LeakyReLU)            │ (None, 392)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │             393 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 923,553 (3.52 MB)

 Trainable params: 923,553 (3.52 MB)

 Non-trainable params: 0 (0.00 B)

También definiremos una red antagónica, que consiste en el generador seguido por el discriminador. Esta red comienza con un vector de ruido y devuelve un resultado binario.


In [6]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim))
        generated_images = self.generator(random_latent_vectors)
        combined_images = tf.concat([generated_images, real_images], axis=0)
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))],
            axis=0
        )
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim))

        misleading_labels = tf.zeros((batch_size, 1))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(
                self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(
            zip(grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {"d_loss": self.d_loss_metric.result(),
                "g_loss": self.g_loss_metric.result()}

In [15]:
import numpy as np

class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images = generated_images.numpy()

        for i in range(self.num_img):
            img = generated_images[i]

            # Add a channel dimension for grayscale image
            if img.shape[-1] != 1:  # Ensure we are adding the correct dimension
                img = np.expand_dims(img, axis=-1)

            img = keras.utils.array_to_img(img, scale=False)
            img.save(f"generated_img_{epoch:03d}_{i}.png")

## Carga del conjunto de datos

Utilizaremos el conjunto de datos MNIST.


In [10]:
(X_train, _), (_, _) = keras.datasets.mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Entrenamiento de la red

En cada paso del entrenamiento, tenemos dos fases:

- Entrenamiento del discriminador:
  - Generamos algunos vectores aleatorios (ruido). El entrenamiento se realiza en minibatches, por lo que usamos batch//2 vectores para producir batch//2 imágenes generadas.
        
  - Seleccionamos batch//2 imágenes aleatorias del conjunto de datos.
        
  - Entrenamos el discriminador con un 50% de imágenes reales y un 50% de imágenes generadas, proporcionando las etiquetas correspondientes (0 o 1).
        
- Entrenamos el generador utilizando el modelo antagónico combinado, pasando vectores aleatorios como entrada y esperando obtener 1's como salida (lo que corresponde a imágenes reales).



In [16]:
epochs = 100

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    X_train, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)

Epoch 1/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 105s 54ms/step - d_loss: 0.2992 - g_loss: 3.4575
Epoch 2/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 139s 53ms/step - d_loss: 0.2255 - g_loss: 3.5753
Epoch 3/100
1088/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 54ms/step - d_loss: 0.2321 - g_loss: 3.4567

KeyboardInterrupt: 

## DCGAN

La **GAN convolucional profunda** (DCGAN, por sus siglas en inglés) es una idea bastante obvia que implica el uso de capas convolucionales tanto para el generador como para el discriminador. La principal diferencia aquí es el uso de la capa **Conv2DTranspose** o **Upsampling** en el generador.


![Imagen de la arquitectura de DCGAN](https://drive.google.com/uc?id=1JejhsoWOTsRVjrl8fy4xJKFNlbNuH3Og)

In [25]:
laten_dim = 128

generator = Sequential(
    [
        # Input layer with 128-dimensional latent space
        Input(shape=(laten_dim,)),

        # First Dense layer to project to 128 * 7 * 7 and reshape to (7, 7, 128)
        Dense(128 * 7 * 7, activation="relu"),
        Reshape((7, 7, 128)),

        # First upsampling and Conv2DTranspose block
        UpSampling2D(),  # Upsample to (14, 14, 128)
        Conv2DTranspose(128, kernel_size=3, padding="same"),
        BatchNormalization(momentum=0.8),
        Activation("relu"),

        # Second upsampling and Conv2DTranspose block
        UpSampling2D(),  # Upsample to (28, 28, 128)
        Conv2DTranspose(64, kernel_size=3, padding="same"),
        BatchNormalization(momentum=0.8),
        Activation("relu"),

        # Final Conv2DTranspose layer to get a single-channel output (grayscale image)
        Conv2DTranspose(1, kernel_size=3, padding="same"),
        Activation("tanh")  # Output activation for GANs (to ensure values between -1 and 1)
    ],
    name = 'generator'
)

generator.summary()

Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_19 (Dense)                     │ (None, 6272)                │         809,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_5 (Reshape)                  │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_4 (UpSampling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_10                  │ (None, 14, 14, 128)         │         147,584 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 14, 14, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_7 (Activation)            │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_5 (UpSampling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_11                  │ (None, 28, 28, 64)          │          73,792 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_8 (Activation)            │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_12                  │ (None, 28, 28, 1)           │             577 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_9 (Activation)            │ (None, 28, 28, 1)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,031,809 (3.94 MB)

 Trainable params: 1,031,425 (3.93 MB)

 Non-trainable params: 384 (1.50 KB)

In [26]:
discriminator = Sequential(
    [
        # Input layer and first Conv2D block
        Input(shape=(28, 28, 1)),
        Conv2D(32, kernel_size=3, strides=2, padding="same"),
        LeakyReLU(negative_slope=0.2),
        Dropout(0.25),

        # Second Conv2D block
        Conv2D(64, kernel_size=3, strides=2, padding="same"),
        ZeroPadding2D(padding=((0, 1), (0, 1))),  # Padding for maintaining dimensions
        BatchNormalization(momentum=0.8),
        LeakyReLU(negative_slope=0.2),
        Dropout(0.25),

        # Third Conv2D block
        Conv2D(128, kernel_size=3, strides=2, padding="same"),
        BatchNormalization(momentum=0.8),
        LeakyReLU(negative_slope=0.2),
        Dropout(0.25),

        # Fourth Conv2D block
        Conv2D(256, kernel_size=3, strides=1, padding="same"),
        BatchNormalization(momentum=0.8),
        LeakyReLU(negative_slope=0.2),
        Dropout(0.25),

        # Flatten and output layer
        Flatten(),
        Dense(1, activation='sigmoid')
    ],
    name = 'discriminator'
)

discriminator.summary()

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 14, 14, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_22 (LeakyReLU)           │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 7, 7, 64)            │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ zero_padding2d_3 (ZeroPadding2D)     │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (None, 8, 8, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_23 (LeakyReLU)           │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_25               │ (None, 4, 4, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_24 (LeakyReLU)           │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 4, 4, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_26               │ (None, 4, 4, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_25 (LeakyReLU)           │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 1)                   │           4,097 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 393,729 (1.50 MB)

 Trainable params: 392,833 (1.50 MB)

 Non-trainable params: 896 (3.50 KB)

In [31]:
X_train = np.expand_dims(X_train, axis=-1)

In [ ]:
epochs = 100

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    X_train, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)

Epoch 1/100
 258/1875 ━━━━━━━━━━━━━━━━━━━━ 19:05 708ms/step - d_loss: 0.2697 - g_loss: 8.9095

> **Tareas**:
- Intenta generar imágenes a color más complejas con DCGAN. Por ejemplo, utiliza una clase del conjunto de datos [CIFAR-10](https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html).

- Cambiar la convolución transpuesta por Upsampling.

- Modificar la función de costo según lo visto en clase y comparar el resultado del entrenamiento.


## Training on Paintings

One of the good candidates for GAN training are paintings created by human artists.

![Imagen de la arquitectura de DCGAN](https://drive.google.com/uc?id=1lYH-kALyjT_PRhOfkS8FKgc-S4vzTMkQ)